# Testing Kalman filter performance

This notebook will run the Kalman filter from `pytpc` for a variety of simulated input tracks to see how it works under different conditions.

In [ ]:
%matplotlib inline

In [ ]:
from IPython.parallel import Client

In [ ]:
rc = Client()
lbv = rc.load_balanced_view()
dv = rc[:]

In [ ]:
%%px --local
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random
import os
from IPython.display import clear_output
import pickle

In [ ]:
sns.set_style('white')

In [ ]:
sns.set_context('notebook', font_scale=1.2)

In [ ]:
%%px --local
import sys
sys.path.append('/Users/josh/Documents/Code/pytpc/')

In [ ]:
%%px --local
import pytpc
from pytpc.constants import *

In [ ]:
from mpl_toolkits.axes_grid.anchored_artists import AnchoredText

---

# Parameters

These are universal and won't be varied in each run.

In [ ]:
%%px --local
tilt = 7*degrees
clock = 6.25
shape = 332
vd_mag = -2.8

emag = 15e3
bmag = 0.5
ef, bf = pytpc.utilities.create_fields(emag, bmag, tilt)

vd = pytpc.simulation.drift_velocity_vector(vd_mag, emag, bmag, tilt)

pads = pytpc.generate_pad_plane()

---

# Definitions

Functions for batch running

In [ ]:
%%px --local
def do_simulate(pmass, pchg, pen, pazi, ppol):
    proj = pytpc.Particle(pmass, pchg, pen, position=(0, 0, 1.0), azimuth=pazi, polar=ppol)
    gas = pytpc.gases.InterpolatedGasMixture(200., ('helium', 0.9), ('carbon_dioxide', 0.1))

    simres = pytpc.track(proj, gas, ef, bf)
    part = simres[simres.en >= 1.0]

    evt = pytpc.evtdata.make_event(part[['x', 'y', 'z']].values * 1e3, part['de'].values, clock, vd, 40., 
                                   proj.mass_num, shape, offset=0)
    
    return part, evt

In [ ]:
%%px --local
def do_filter(evt, guess_A, guess_Z, guess_en):
    meas = evt.xyzs(vd, clock, peaks_only=True, pads=pads)[:, 0:3] * 1e-3
    meas = meas[meas[:, 2].argsort()][::-1]
    
    guess_diff = meas[1:] - meas[0]
    guess_azi = np.median(np.arctan2(guess_diff[:, 1], guess_diff[:, 0]))
    guess_pol = np.median(np.arctan2(np.sqrt(guess_diff[:, 0]**2 + guess_diff[:, 1]**2), guess_diff[:, 2]))
    
    trproj = pytpc.Particle(guess_A, guess_Z, guess_en, position=meas[0], azimuth=guess_azi, polar=guess_pol)
    gas = pytpc.gases.InterpolatedGasMixture(200., ('helium', 0.9), ('carbon_dioxide', 0.1))

    tr = pytpc.Tracker(trproj, gas, ef, bf, trproj.state_vector)
    tr.kfilter.Q = np.diag((1e-4, 1e-4, 1e-4, 1e-1, 1e-1, 1e-1))**2 # Process
    tr.kfilter.R = np.diag([2e-2]*2 + [4e-2]) ** 2 # Measurement
    tr.kfilter.P = np.diag([1e-2] * 3 + [1e-0] * 3)**2

    res, covar, res_times = tr.track(meas)

    resdf = pd.DataFrame(res, columns=('x', 'y', 'z', 'px', 'py', 'pz'))
    resdf['time'] = res_times
    resdf['en'] = (np.sqrt(np.sum(res[:, 3:6]**2, axis=1) + trproj.mass**2) - trproj.mass) / trproj.mass_num
    resdf['de'] = np.concatenate((np.array([0.0]), np.diff(resdf.en)))
    resdf['azi'] = np.arctan2(resdf.py, resdf.px)
    resdf['pol'] = np.arctan2(np.sqrt(resdf.px**2 + resdf.py**2), resdf.pz)
    
    return resdf

In [ ]:
%%px --local
def run_event(pvec):
    try:
        part, evt = do_simulate(pvec.mass_in, pvec.chg_in, pvec.en_in, pvec.azi_in, pvec.pol_in)
        resdf = do_filter(evt, pvec.mass_in, pvec.chg_in, pvec.guess_en)
        lastpt = resdf.iloc[-1].copy()
        truept = part.iloc[-1].copy()
        truept.index = ['true_' + x for x in truept.index]
        truept['start_en'] = pvec.en_in
    except Exception as e:
        print(type(e), e)
        pass
    else:
        return pd.concat((lastpt, truept))

In [ ]:
def stats_box(dist, loc, size):
    s = 'Mean: {m:0.4f}\nSD: {sd:0.4f}\nN: {n}'.format(m=dist.mean(), 
                                                       sd=dist.std(), 
                                                       n=len(dist))
    at = AnchoredText(s, loc=loc, prop={'size': size})
    return at

---

# Simulation of events

Now I'll run the code for a lot of events to see how it responds. That's done in the next few cells.

First, I'll set the parameters for running. I chose to use normally distributed beam energies with mean 2.0 MeV/u and standard deviation 0.1 MeV/u. The azimuthal energy and polar energy were each uniformly distributed over all angles. (Note that I did not include backwards polar angles since I started the track at the cathode of the detector, and these would immediately leave the detector volume.)

Another note: I chose to always seed the Kalman filter with the same value of energy, which was chosen to be the mean of the normally distributed energies (2.0 MeV/u). I did this because this is how we will probably have to run it in real life, where we only know the mean beam energy.

In [ ]:
niters = 10000
params = pd.DataFrame(0, columns=('mass_in', 'chg_in', 'en_in', 'azi_in', 'pol_in', 'guess_en'), 
                      index=range(niters))
params.mass_in = 4
params.chg_in = 2
params.en_in = np.random.normal(2, 0.1, niters)
params.azi_in = np.random.uniform(0, 2*pi, niters)
params.pol_in = np.random.uniform(pi/2, pi, niters)
params.guess_en = 2.

This actually runs the simulation for all of the events and saves the results. This takes about 20 minutes for 10000 events running on 6 IPython engines.

In [ ]:
res = dv.map_sync(lambda x: run_event(x[1]), params.iterrows())
resdf = pd.DataFrame({i: v for i, v in enumerate(res) if v is not None}).transpose()
mcdf = pd.concat((params, resdf), axis=1)
mcdf['pdev_en'] = (mcdf.en - mcdf.true_en) / mcdf.true_en * 100
mcdf['dev_azi'] = (mcdf.azi - mcdf.true_azi)
mcdf['dev_pol'] = (mcdf.pol - mcdf.true_pol)
mcdf['en_err'] = mcdf.start_en - 2.0
mcdf.to_pickle('new_mcdf.p')
with open('new_mcdf.txt', 'w') as f:
    f.write(mcdf.to_string())

The next cell just reads back in the data from the file.

In [ ]:
mcdf = pd.read_pickle('new_mcdf.p')

# Cut for events that leave the detector

I stopped each track at 1.0 MeV/u, choosing this energy arbitrarily to be like a reaction vertex. This poses problems for the high-polar-angle tracks that leave the detector before that energy. Therefore, I chose to cut these out.

Here is the distribution of true "reaction vertex" energies before the cut (i.e. the energy of the particle at the last point of the simulated track):

In [ ]:
plt.hist(mcdf.true_en.dropna(), bins=20);
plt.xlabel('True vertex energy [MeV/u]')
plt.ylabel('Count');

This can be compared to the polar angle to see that the odd hump in the middle is due to tracks at very horizontal polar angles. Keep in mind that in my coordinate system (see below), the window is at (0, 0, 1) meters, so a polar angle of π is straight ahead.

In [ ]:
# Show the coordinate system image
from IPython.display import Image
Image('CoordinateSystems.png', width=300)

The plot below shows the comparison between vertex energy and polar angle.

In [ ]:
sns.jointplot('pol_in', 'true_en', mcdf, kind='hex', stat_func=None)
plt.sca(plt.gcf().axes[0])
plt.xlabel('Input polar angle [radians]')
plt.ylabel('True vertex energy [MeV/u]');

Clearly this is not ideal. Since they should always be around 1.0 MeV/u, I'll cut out the strange ones.

In [ ]:
reac_cut = mcdf[mcdf.true_en < 1.003].dropna()
plt.hist(reac_cut.true_en, bins=20);

---

# Results

The results are pretty good.

### Azimuthal Angle

First, let's look at the deviation in the azimuthal angle, i.e. the difference between the measured and true azimuthal angles. This distribution forms a spike near 0 degrees.

In [ ]:
plt.hist(reac_cut.dev_azi / degrees, bins=100, histtype='bar')
# plt.grid()
# sns.distplot(reac_cut.dev_azi, kde=False, fit=scipy.stats.norm)
plt.xlabel('Deviation in azimuthal angle [degrees]')
plt.ylabel('Count')
at = stats_box(reac_cut.dev_azi / degrees, 2, 12)
plt.gca().add_artist(at);

There are a few points at each end near 360 degrees, but those might be due to a branch cut or something.

### Polar Angle

This one has a very narrow distribution (notice how small the scale is on the x axis).

In [ ]:
sns.distplot(reac_cut.dev_pol / degrees, kde=False, fit=scipy.stats.norm)
plt.xlabel('Deviation in polar angle [degrees]')
plt.ylabel('Count')
at = stats_box(reac_cut.dev_pol / degrees, 2, 12)
plt.gca().add_artist(at);

### Vertex energy

This variable has the largest spread of any of them. 

In [ ]:
sns.distplot(reac_cut.en_err, kde=False, fit=scipy.stats.norm)
plt.xlabel('Deviation in vertex energy [MeV/u]')
plt.ylabel('Count')
at = stats_box(reac_cut.en_err, 2, 12)
plt.gca().add_artist(at);

This can be explained completely by the assumption I made for the Kalman filter, however. Rememeber that I always told the Kalman filter that the beam energy started at 2.0 MeV/u. If we compare the deviation to the actual beam energy, they are extremely correlated:

In [ ]:
sns.jointplot(reac_cut.en_in, reac_cut.en_err, kind='reg', marginal_kws={'kde': False})
plt.xlabel('Beam energy [MeV/u]')
plt.ylabel('Deviation in vertex energy [MeV/u]')

### Interaction of variables

Finally, we can see that most of the variables are not correlated to each other. The plot below shows a kernel density estimation for each set of variables.

In [ ]:
pg = sns.PairGrid(reac_cut, x_vars=('en_in', 'azi_in', 'pol_in'), y_vars=('en_err', 'dev_azi', 'dev_pol'),
                  despine=False)
pg.map(sns.kdeplot, shade=True, cmap='Blues')

**Note**: the axis labels here are odd since it's hard to change them on this plot. The ones labeled `[something]_in` are the input parameters to the simulation; the ones with `dev_[something]` are the deviations (differences); and `en_err` is the deviation in the vertex energy. All of the angles are in radians.